In [1]:
import models

c:\Users\aruav\Desktop\Deepen\CTG\cambio\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
models.predict_ipca()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=300` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=300` reached.


,prediction,superior,inferior,time,indicador
2022-10,6370.401591,6785.069507,5955.733676,2022-12-14 11:45:05.717038,IPCA
2022-11,6535.250233,6960.648624,6109.851841,2022-12-14 11:45:05.717038,IPCA
2022-12,6691.545066,7127.117141,6255.972991,2022-12-14 11:45:05.717038,IPCA
2023-01,6624.843325,7056.073591,6193.613060,2022-12-14 11:45:05.717038,IPCA
2023-02,6486.879573,6909.129378,6064.629769,2022-12-14 11:45:05.717038,IPCA
2023-03,6660.918425,7094.496924,6227.339927,2022-12-14 11:45:05.717038,IPCA
2023-04,6652.609619,7085.647273,6219.571965,2022-12-14 11:45:05.717038,IPCA
2023-05,6481.179356,6903.058117,6059.300595,2022-12-14 11:45:05.717038,IPCA
2023-06,6664.666482,7098.488952,6230.844012,2022-12-14 11:45:05.717038,IPCA
2023-07,6550.765838,6977.174185,6124.357491,2022-12-14 11:45:05.717038,IPCA
